In [1]:
import re
import numpy as np
import pandas as pd
import seaborn as sns

from pathlib import Path
from scipy import stats
from veliadb import base, settings
from veliadb import benchling_orm as bo
from veliadb.base import (Session, Orf, OrfXref, Transcript, Gene, 
                          TranscriptOrf, SequenceRegionXref, Protein, 
                          ProteinXref, Dataset, ProteinOrf)

from sqlalchemy.sql.expression import func, and_, or_

from dashboard import data_load
import pyarrow.parquet as pq
from dashboard.etl import CACHE_DIR, TPM_DESEQ2_FACTOR, DATA_DIR

pd.options.display.max_columns = 50
pd.options.display.max_colwidth = 100

2024-05-25 05:12:19.169 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-25 05:12:19.171 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-25 05:12:19.172 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-25 05:12:19.174 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-25 05:12:19.176 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-25 05:12:19.177 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-25 05:12:19.178 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-25 05:12:19.179 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-

In [2]:
session = base.Session()

In [3]:
session.query(Orf).count()

856944

In [3]:
dashboard_orfs = session.query(Orf).filter(Orf.vtx_id != '').all()

In [5]:
len(dashboard_orfs)

16837

In [12]:
len(set([d.vtx_id for d in dashboard_orfs]))

16837

In [14]:
with open('../cache_updates/all_vtx_240524.txt', 'w') as outfile:
    for orf in dashboard_orfs[0:100]:
        outfile.write(f'{orf.vtx_id}\n')


### Swissprot

In [35]:
swissprot_query = \
    session.query(Protein, Orf)\
           .join(ProteinXref, ProteinXref.protein_id == Protein.id)\
           .join(Dataset, Dataset.id == ProteinXref.xref_dataset_id)\
           .join(ProteinOrf, ProteinOrf.protein_id == Protein.id)\
           .join(Orf, Orf.id == ProteinOrf.orf_id)\
           .filter(Dataset.name == 'swissprot')\
           .filter(func.length(Protein.aa_seq) < 150)\
           .distinct(ProteinXref.protein_id)



In [32]:
alan_df = pd.read_csv('../../data/orfs_under_150codons_with_function-2.csv')

In [37]:
with open('../cache_updates/swissprot_sORF_150aa_vtx.txt', 'w') as outfile:
    for prot, orf in swissprot_query.all():
        outfile.write(f'VTX-{orf.id:07d}\n')
    

In [28]:
prot, orf = swissprot_query.first()

In [19]:
prot.

SyntaxError: invalid syntax (2621462385.py, line 1)

In [15]:
prot.ensembl_protein_id

'ENSP00000319240.2'

In [14]:
orf.ensembl_protein_id

'ENSP00000319240.2'

In [12]:
session.query(ProteinOrf).count()

187319